In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField

In [5]:
bdname='sinesearch_db'
bduser = 'postgres'
bdpassword = 'nef441'
bdhost = '51.79.69.179'
bdport = 5432

db = PostgresqlExtDatabase(bdname, user=bduser, password=bdpassword,
                           host=bdhost, port=bdport)

In [6]:
class Users(Model):
    tel_id=IntegerField(index=True)
    vk_id=IntegerField()
    name = TextField()
    nicname = TextField(null=True)
    ustatus=IntegerField(default=0)
    age=IntegerField(default=20)
    sex=IntegerField(default=0)
    ganres=JSONField(default={})
    info=JSONField(default={})
    last_visit=DateField(null=True)
    mark_wight=FloatField(default=0)
    just_marked = IntegerField(default=1)

    cms=IntegerField(default=0)
    cfid=IntegerField(default=0)
    cfmes=IntegerField(default=0)
    ctmes=IntegerField(default=0)
    liked=ArrayField(IntegerField,default=[])
    viewed=ArrayField(IntegerField,default=[])
    disliked=ArrayField(IntegerField,default=[])
    predict_films=ArrayField(IntegerField,default=[])
    selections=JSONField(default={})

    class Meta:
        database = db
        db_table='Users'

class Films(Model):
    film_id=IntegerField(unique=True,index=True)
    name = TextField()
    discr = TextField(null=True)
    img = TextField(null=True)
    youtube=TextField(default="")
    stars = FloatField(default=0)
    info=JSONField(default={})
    level=IntegerField(default=0)
    selections=ArrayField(IntegerField,default=[])
    year=IntegerField(default=2000)

    ganres=ArrayField(IntegerField,default=[])
    meanage=FloatField(default=20)
    sex=FloatField(default=0.5)

    likes=IntegerField(default=10)
    dislikes=IntegerField(default=10)
    shit=IntegerField(default=0)
    errors=IntegerField(default=0)
    opening=IntegerField(default=0)
    treilers=IntegerField(default=0)

    class Meta:
        database = db
        db_table='Films'

class Selections(Model):
    name = TextField()
    user_id = IntegerField(default=0)
    stars = FloatField(default=0)
    films = ArrayField(IntegerField,default=[])

    class Meta:
        database = db
        db_table='Selections'

class Messages(Model):
    mes_id = TextField(unique=True)
    text = TextField(default="")
    reply_markup = JSONField(default={})
    user = IntegerField(default=0)
    btime = DateField(null=True)

    class Meta:
        database = db
        db_table='Messages'

class Config(Model):
    name = TextField()
    value = TextField()
    json=JSONField(default={})

    class Meta:
        database = db
        db_table='Config'

class Dataset(Model):
    user_value = JSONField()
    film_value = JSONField()
    result = IntegerField(default=0)
    data = DateField()

    class Meta:
        database = db
        db_table='Dataset'

# Edit Constants

In [10]:
name=input("Name: ")
val=input("Value: ")
Config.update({"value":val}).where(Config.name==name).execute()

Name: WEIGHT_YEAR
Value: 5


1

In [8]:
Films.update({Films.stars:(Films.likes*10+Films.dislikes*3+Films.shit*1)/(Films.likes+Films.dislikes+Films.shit)}).where(Films.stars>10).execute()

0

# Server Information

In [22]:
from datetime import datetime,timedelta
dat = datetime.now() - timedelta(days=2)

info={
    "Users":Users.select().count(),
    "Films":Films.select().count(),
    "Messages":Messages.select().count(),
    "Dataset":Dataset.select().count(),
    "Dataset Day":Dataset.select().where(Dataset.data>dat).count(),

}
from IPython.core.display import HTML
html=""
for i in info:
    html+="<b style=\"width=300px;\">"+i+"</b>   "+str(info[i])+"</br>"
HTML(html)

In [23]:
CONFIG = {}
for c in Config.select():
    if c.value == "json":
        CONFIG[c.name] = c.json
    else:
        CONFIG[c.name] = c.value

In [24]:
import telebot
from telebot import types
bot = telebot.TeleBot(CONFIG["TELEGRAM_TOKEN"])


In [25]:
text="Нас уже больше 100 человек! рады сообщить, что вчара было произведено первое дообучение бота на данных пользователей (за 3 дня вы разметили более 1200 фильмов). Следующее дообучение будет при достижении 300 пользователей, если мы достигнем этого числа до понедельника, в боте появится раздел с новинками кино.\nДля этого надо рассказать о боте всего 2ум людям, это кудп роще чем кажется. Как только мы наберем 1000 пользователей качество должно достигнуть 6 кликов до нужного фильма!"

In [27]:
for u in Users.select().execute():
    try:
        bot.send_message(chat_id=u.tel_id,
                             parse_mode="HTML",
                             text=text)
    except:
        print(u.name)

Melissa None
Pavel Zolotov
Valeriy Mironov
🐦 None
